In [2]:
import pandas as pd
import snowflake.connector
from snowflake.snowpark import Session
import time
import json
from datetime import datetime

In [3]:
# Snowflake connection parameters
snowflake_config = {
    "account": "YMBZKCC-NT32058",  # e.g., "xyz12345.us-east-1"
    "user": "UMI",
    "password": "Umair2002121212",
    "warehouse": "COMPUTE_WH",
    "database": "AGRICULTURE_DB",
    "schema": "DATA_ANALYSIS",
    "role": "ACCOUNTADMIN"
}

In [4]:
# Initialize Snowflake connection and session
conn = snowflake.connector.connect(**snowflake_config)
session = Session.builder.configs(snowflake_config).create()

print("Snowflake connection established!")

Snowflake connection established!


In [5]:
# Create database and schema
session.sql("CREATE DATABASE IF NOT EXISTS AGRICULTURE_DB").collect()
session.sql("USE DATABASE AGRICULTURE_DB").collect()
session.sql("CREATE SCHEMA IF NOT EXISTS DATA_ANALYSIS").collect()
session.sql("USE SCHEMA DATA_ANALYSIS").collect()

# Create table for streaming data
create_table_query = """
CREATE OR REPLACE TABLE agriculture_data_stream (
    Year INT,
    Location STRING,
    Area FLOAT,
    Rainfall FLOAT,
    Temperature FLOAT,
    Soil_type STRING,
    Irrigation STRING,
    Yields FLOAT,
    Humidity FLOAT,
    Crops STRING,
    Price FLOAT,
    Season STRING,
    Ingestion_Time TIMESTAMP
);
"""
session.sql(create_table_query).collect()

# Create stream for real-time processing
session.sql("CREATE OR REPLACE STREAM agriculture_stream ON TABLE agriculture_data_stream").collect()

print("Database, schema, table, and stream set up successfully.")

Database, schema, table, and stream set up successfully.


In [ ]:
# Load your dataset
from datetime import timezone
import pandas as pd
from datetime import datetime
import time

df = pd.read_csv("C:/Users/moham/Downloads/data_season.csv")  # Ensure this file is in your directory

# Streaming function with error handling
def stream_to_snowflake(row):
    try:
        row_dict = row.to_dict()
        row_dict['Ingestion_Time'] = datetime.now(timezone.utc).isoformat()
        insert_query = f"""
        INSERT INTO agriculture_data_stream
        VALUES ({row_dict['Year']}, '{row_dict['Location']}', {row_dict['Area']},
                {row_dict['Rainfall']}, {row_dict['Temperature']}, '{row_dict['Soil type']}',
                '{row_dict['Irrigation']}', {row_dict['Yeilds']}, {row_dict['Humidity']},
                '{row_dict['Crops']}', {row_dict['Price']}, '{row_dict['Season']}',
                TO_TIMESTAMP('{row_dict['Ingestion_Time']}'))
        """
        session.sql(insert_query).collect()
    except Exception as e:
        print(f"Error streaming row: {e}")

# Continuous simulation (loop indefinitely)
print("Starting continuous streaming...")
while True:
    for index, row in df.iterrows():
        stream_to_snowflake(row)
        print(f"Streamed row {index + 1}/{len(df)} at {datetime.now(timezone.utc)}")
        time.sleep(0.1)
    print("Cycle complete. Restarting...")

Starting continuous streaming...
Streamed row 1/3100 at 2025-03-25 21:45:50.531079+00:00
Streamed row 2/3100 at 2025-03-25 21:45:51.187442+00:00
Streamed row 3/3100 at 2025-03-25 21:45:51.674714+00:00
Streamed row 4/3100 at 2025-03-25 21:45:52.107056+00:00
Streamed row 5/3100 at 2025-03-25 21:45:52.640934+00:00
Streamed row 6/3100 at 2025-03-25 21:45:53.073063+00:00
Streamed row 7/3100 at 2025-03-25 21:45:53.589136+00:00
Streamed row 8/3100 at 2025-03-25 21:45:54.089446+00:00
Streamed row 9/3100 at 2025-03-25 21:45:54.588191+00:00
Streamed row 10/3100 at 2025-03-25 21:45:55.039496+00:00
Streamed row 11/3100 at 2025-03-25 21:45:55.523134+00:00
Streamed row 12/3100 at 2025-03-25 21:45:55.988058+00:00
Streamed row 13/3100 at 2025-03-25 21:45:56.410567+00:00
Streamed row 14/3100 at 2025-03-25 21:45:56.981274+00:00
Streamed row 15/3100 at 2025-03-25 21:45:57.804382+00:00
Streamed row 16/3100 at 2025-03-25 21:45:58.257711+00:00
Streamed row 17/3100 at 2025-03-25 21:45:58.755772+00:00
Streame

In [14]:
session.sql("TRUNCATE TABLE agriculture_data_stream").collect()
print("Table truncated. Starting fresh streaming.")

Table truncated. Starting fresh streaming.
